# Macros 

So far we have been creating Julia expressions ourselves, by hand. Macros are a common method to do this automatically.

Recall that macros begin with `@` and behave like "super-functions", which take in a piece of code and replace it with another piece of code.
The effect of a macro call is to splice, or replace, the new piece of code in place of the old code; the new code is what is actually compiled by the Julia compiler. 

Note that the user *does not need to explicitly pass an `Expr`ession object*; Julia turns the code that follows the macro call into an expression.

To see this, let's define the simplest macro:

In [1]:
macro simple(ex)
    @show typeof(ex)
    @show ex
    
    return nothing   # return nothing for the moment
end

@simple (macro with 1 method)

and run it:

In [5]:
@simple 1

typeof(ex) = Int64
ex = 1


In [6]:
@simple x + y

typeof(ex) = Expr
ex = :(x + y)


In [8]:
macro simple2(ex)
    return ex
end

@simple2 (macro with 1 method)

In [9]:
@simple2 x + y

LoadError: UndefVarError: x not defined

We see that the code `a = b` has *automatically been captured as a Julia expression*. Basically the *string* `"a = b"` that we typed has been **parsed**, i.e. converted into Julia code. We can do this ourselves:

In [6]:
Meta.parse("a = b")

:(a = b)

The macro can now take this `Expr` object and process it as we have seen in the previous notebooks. The macro will return a new `Expr`; this is the new code that will actually be compiled instead of the old code!

## Structuring a macro

The usual recommendation is that a macro should just act as an interface to the user that captures the user's code, as we saw just above. The resulting expression is then usually passed to a function to do the manipulation. This gives a separation of concerns (capture vs. processing) and makes it easier for the developer to test the processing step.

For example, let's write a macro that replaces a `+` with a `*`:

In [10]:
macro add_to_multiply(ex)
    
    return _add_to_multiply(ex)
end

@add_to_multiply (macro with 1 method)

In [11]:
@add_to_multiply x + y

LoadError: LoadError: UndefVarError: _add_to_multiply not defined
in expression starting at In[11]:1

The function receives an expression and should create the new expression:

In [12]:
function _add_to_multiply(ex)
    
    if ex.head == :call && ex.args[1] == :+
        ex.args[1] = :*
    end
        
    return ex
end

_add_to_multiply (generic function with 1 method)

In [13]:
_add_to_multiply( :(a + b) )

:(a * b)

In [14]:
_add_to_multiply( :(a - b) )

:(a - b)

In [15]:
@add_to_multiply a + b

LoadError: UndefVarError: a not defined

What is happening here? The macro *first* replaces the code with `a * b`, and *then* compiles and tries to execute the code. 
But the variable `a` does not yet exist, so this errors. Defining `a` and `b` behaves as we expect:

In [16]:
a = 2; b = 3

3

In [17]:
@add_to_multiply a + b

6

We indeed get the result of multiplying `a` and `b`, not adding them.

This is a good example of why debugging macros is best done via an intermediate function that does the expression manipulation.

Recall that we can use `@macroexpand` to see what the macro is doing:

In [18]:
@macroexpand @add_to_multiply a + b

:(Main.a * Main.b)

## Macro hygiene

Macro "hygiene" refers to the fact that macros do some modification of the code that they receive, in order to be "hygienic" (clean): they try not to touch user code.

## Exercise

1. Define a macro `@simple3` that returns the expression that was passed to it.


2. What happens when you call `@simple3 yy = xx^2`?


3. Define a variable `xx` with the value `3`. Does the macro work now?


4. Does the variable `yy` now exist?


5. To see what's happening, use `@macroexpand`.

In [22]:
macro simple3(ex)
    return ex
end

@simple3 (macro with 1 method)

In [23]:
@simple3 yy = xx^2

LoadError: UndefVarError: xx not defined

In [24]:
xx

LoadError: UndefVarError: xx not defined

In [26]:
xx = 10

10

In [27]:
@simple2 yy = xx^2

100

In [28]:
yy

LoadError: UndefVarError: yy not defined

In [30]:
@macroexpand @simple3 yy = xx^2

:(var"#62#yy" = Main.xx ^ 2)

You should find that the variable `yy` does *not* now exist, even though it seems like it should, since the code `yy = xx^2` was evaluated. However, macros by default do not "touch" variables in the context from where they are called, since this may have unintended consequences. We refer to this as macro **hygiene**: the macro is **hygienic**, i.e. clean, meaning that it does not "infect" the user's code.

Nonetheless, often we may *want* a macro to be able to modify a variable in the context from which the macro is called.
In which case we can "escape" from this hygiene, making a non-hygienic macro, using `esc` ("escape"):

## Interpolation

In [33]:
ex = :(x + y)

:(x + y)

In [34]:
ex2 = :(z * $(ex))

:(z * (x + y))

In [35]:
macro simple4(ex)
    ex2 = esc(ex)
    
    ex3 = :($(ex2))
    
    return ex3
end

@simple4 (macro with 1 method)

In [37]:
@simple4 yy = xx^2

100

In [38]:
yy

100

In [39]:
@macroexpand @simple4 yy = xx^2

:(yy = xx ^ 2)

Note that once again the macro must return an *expression*. 

For code clarity it is possible to define a new variable that is the escaped version:

In [47]:
macro simple5(ex)
    ex2 = esc(ex)
    
    return ex2
end

@simple5 (macro with 1 method)

In [50]:
@simple5 zz = xx^2

100

In [51]:
zz

100

In [49]:
@macroexpand @simple5 yy = xx^2

:(yy = xx ^ 2)

In [54]:
@macroexpand @simple3 yy = xx^2

:(yy = xx ^ 2)

## Exercise

1. Check that `@simple3` does create a variable `yy`.

## Exercise

Define a `@myshow` macro that reproduces the behaviour of `@show`.

In [40]:
y = 3

3

In [41]:
@show y

y = 3


3

In [52]:
macro myshow(ex)
    return _myshow(ex)
end

@myshow (macro with 1 method)

In [70]:
function _myshow(ex)
    ex2 = QuoteNode(ex)
    
    :( println($ex2, " = ", $ex) )
end

_myshow (generic function with 1 method)

In [62]:
ex = :x

:x

In [63]:
QuoteNode(ex)

:(:x)

In [72]:
_myshow(:x)

:(println(:x, " = ", x))

In [60]:
x = 10

10

In [61]:
@myshow x

10 = 10


In [71]:
@myshow x + y

x + y = 13


In [69]:
y

3

## Exercise 8

1. Write a macro `@replace` that replaces terms in an expression. Apply it to `yy = xx^2 + xx`, replacing `xx` by `xx + 1`.


2. Write a macro `@checked` that replaces all arithmetic operations (`+`, `-`, `*`, `/`) with checked operations (`Base.checked_add` etc.)

# Macros for domain-specific languages

Let's see some simple examples of how we can start to approach domain-specific languages for scientific applications. 

Let's suppose we want to reproduce the `@variables` macro from `Symbolics.jl`. 
The idea is that there is a `Variable` object representing a symbolic variable:

In [73]:
struct Variable
    name::Symbol
end

# Base.show(io::IO, v::Variable) = print(io, v.name)

And we could define arithmetic operations on those to carry out symbolic manipulations (exercise!).

We can create such a variable as

In [74]:
Variable(:x)

Variable(:x)

To define a Julia variable called `x` that is bound to the `Variable` object, we must do

In [75]:
x = Variable(:x)

Variable(:x)

The situation is similar to the `@show` macro: we would ideally like to be able to write `@var x`, which expands to `x = Variable(:x)`.

## Exercise

1. Try to write the `@var` macro. You will probably get stuck! Where is the sticking point?

[Blank!]

In [76]:
macro var(ex)
    return _var(ex)
end

@var (macro with 1 method)

In [79]:
function _var(ex)
    :($(esc(ex)) = Variable($(QuoteNode(ex))))
end

_var (generic function with 1 method)

In [80]:
_var(:x)

:($(Expr(:escape, :x)) = Variable(:x))

In [83]:
@var z

Variable(:z)

In [84]:
z

Variable(:z)

In [85]:
@macroexpand @var z


:(z = Main.Variable(:z))

In [61]:
macro var(ex)
    return _var(ex)
end

@var (macro with 1 method)

In [64]:
@macroexpand @var x

:(var"#65#x" = Main.Variable((Main.Meta).quot(var"#65#x")))

In [147]:
function _var(ex)
    escaped_ex = esc(ex)
    quoted_ex = QuoteNode(ex)
    
    return :(
        $(escaped_ex) = Variable( $(quoted_ex) )
    )
end

_var (generic function with 1 method)

In [126]:
_var(:x)

quote
    #= In[125]:6 =#
    $(Expr(:escape, :x)) = Variable(:x)
end

In [127]:
@macroexpand @var x

quote
    #= In[125]:6 =#
    x = Main.Variable(:x)
end

In [128]:
@macroexpand @var x

quote
    #= In[125]:6 =#
    x = Main.Variable(:x)
end

In [129]:
@var z

z

In [130]:
z

z

In [80]:
y

3

## Several variables

Now suppose we want to expand our macro to handle not only single variables, but also multiple variables, e.g. `@var x y`

The first task is to work out what happens when the macro receives this

In [86]:
macro simple(ex)
    @show ex
    return nothing
end

@simple (macro with 1 method)

In [87]:
@simple x y

LoadError: LoadError: MethodError: no method matching var"@simple"(::LineNumberNode, ::Module, ::Symbol, ::Symbol)
[0mClosest candidates are:
[0m  var"@simple"(::LineNumberNode, ::Module, ::Any) at In[86]:1
in expression starting at In[87]:1

This cryptic error message is telling us that there is no method for the `@simple` macro with *two* arguments (the `::Symbol, ::Symbol`) at the end of the `MethodError`). We need to define a method that can receive two arguments:

In [88]:
macro simple(ex1, ex2)
    @show ex1, ex2
    return nothing
end

@simple (macro with 2 methods)

In [89]:
@simple x y

(ex1, ex2) = (:x, :y)


In general we could have an arbitrary number of arguments; we should pass each through to the `_var` function:`

In [92]:
macro var(exs...)   # varargs (variable number of arguments)
    @show exs
    
    all_code = [_var(ex) for ex in exs]
    
    @show all_code
    
    return :nothing
        
end

@var (macro with 2 methods)

In [93]:
@var x y

exs = (:x, :y)
all_code = Expr[:($(Expr(:escape, :x)) = Variable(:x)), :($(Expr(:escape, :y)) = Variable(:y))]


Now we need to combine that code into a single code block:

In [94]:
macro var(exs...)
    all_code = quote end  # empty block
    
    all_code.args = reduce(vcat, _var(ex) for ex in exs)
    
   #  @show all_code
    
    # return :nothing
    
    return all_code
        
end

@var (macro with 2 methods)

In [97]:
@macroexpand @var x y

quote
    x = Main.Variable(:x)
    y = Main.Variable(:y)
end

### Exercise: 

Rewrite the code so that a list of all the generated variables is returned.

In [99]:
using Symbolics

In [101]:
result = @variables x, y

2-element Vector{Num}:
 x
 y

In [102]:
typeof(result)

Vector{Num} (alias for Array{Num, 1})

In [159]:
@var w t

t

In [160]:
t

t

In [161]:
w

w

## Vectors of variables

Finally let's suppose we want to allow `@var x[1:10]` to make variables called `x1`, ..., `x10`.

The first thing to do is to work out how Julia stores `x[1:10]` as an expression:

In [111]:
x = [Num(Symbolics.Sym{Real}(:x, i)) for i in 1:5]

5-element Vector{Num}:
 x₁
 x₂
 x₃
 x₄
 x₅

In [107]:
vars = @variables x[1:5]
vars = vars[1]

x[1:5]

In [108]:
collect(vars)

5-element Vector{Num}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]

In [162]:
Meta.parse("x[1:10]")

:(x[1:10])

In [163]:
dump(ans)

Expr
  head: Symbol ref
  args: Array{Any}((2,))
    1: Symbol x
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol :
        2: Int64 1
        3: Int64 10


We see that the answer is that the `head` of the `Expr` is `ref`.

## Exercise

1. Write a function to deal with this case and generate the variables

2. Modify the macro to do so

## Extra trick:

Thanks to Simeon Schaub for the following trick: 

You can call a macro in the following way to get back the expression:

In [166]:
var"@simple"(LineNumberNode(3), Main, x, y)

(ex1, ex2) = (x, y)
